In [2]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *


done1
done2


In [3]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'07082017'

script outline:
1. define which samples belong to the test and which to the train sets (modular- enable definition of sample numbers of each set)
2. calculate probs for each event seperately in the train and test sets 
3. correlate results from the two halfs
4. consider repeating several times to see consistency.
5. from one half, predict how many shared samples will be in the other half, and vice versa, and compare to the real result. 
6. leave1out analysis in the whole sequence level?
7. predict productive sequences freqs according to the non-prod calculations? (maybe start from the ins1 data first...)

In [5]:
def divide_sampleList_into_TrainAndTest(nSamplesInTest):
    
    df_file_names,samples_with_df=get_sample_with_dfs()
    
    import random
    ShuffledSampleList=samples_with_df
    random.shuffle(ShuffledSampleList)
    
    
    TestSetSamples = ShuffledSampleList[:nSamplesInTest]
    trainSetSamples = ShuffledSampleList[nSamplesInTest:]
    
    return TestSetSamples, trainSetSamples

In [45]:
587./2

293.5

In [6]:

TestSetSamples, trainSetSamples=divide_sampleList_into_TrainAndTest(293)

getting list of samples that have dfs...
finished getting list of samples that have dfs


In [7]:
print len(trainSetSamples)

294


In [8]:
print len(TestSetSamples)

293


In [10]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/trainSetSamples','wb') as f1:
    pickle.dump(trainSetSamples,f1)
f1.close()

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/TestSetSamples','wb') as f2:
    pickle.dump(TestSetSamples,f2)
f2.close()



In [12]:
#generate v and j lists for the train and test sets:

vGeneTrainSetDF_list = []
jGeneTrainSetDF_list = []
vGeneTestSetDF_list = []
jGeneTestSetDF_list = [] 
    
for sample_name in trainSetSamples:
    
    print 'trainSetSamples'
    print sample_name
    
    sample_df = pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/%s.tsv" % sample_name) 
    sample_df_non_prod = sample_df[sample_df['sequenceStatus'] != 'In']
    
        
    vGeneTrainSetDF = pd.DataFrame(sample_df_non_prod['vGeneName'])
    jGeneTrainSetDF = pd.DataFrame(sample_df_non_prod['jGeneName'])   
    vGeneTrainSetDF['Sample'] = sample_name
    jGeneTrainSetDF['Sample'] = sample_name  
    vGeneTrainSetDF_list.append(vGeneTrainSetDF)
    jGeneTrainSetDF_list.append(jGeneTrainSetDF)
    
    
for sample_name in TestSetSamples:
    
    print 'TestSetSamples'
    print sample_name    
    
    sample_df = pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/%s.tsv" % sample_name) 
    sample_df_non_prod = sample_df[sample_df['sequenceStatus'] != 'In']

    vGeneTestSetDF = pd.DataFrame(sample_df_non_prod['vGeneName'])
    jGeneTestSetDF = pd.DataFrame(sample_df_non_prod['jGeneName'])   
    vGeneTestSetDF['Sample'] = sample_name
    jGeneTestSetDF['Sample'] = sample_name  
    vGeneTestSetDF_list.append(vGeneTestSetDF)
    jGeneTestSetDF_list.append(jGeneTestSetDF)

AllvGeneTrain = pd.concat(vGeneTrainSetDF_list)
AlljGeneTrain = pd.concat(jGeneTrainSetDF_list)
AllvGeneTest = pd.concat(vGeneTestSetDF_list)
AlljGeneTest = pd.concat(jGeneTestSetDF_list)

#print len(AllvGeneDFs)

AllvGeneTrainsmall = AllvGeneTrain.set_index('Sample')
AllvGeneTrainsmall.index = AllvGeneTrainsmall.index.str.replace("HIP", "")

AlljGeneTrainsmall = AlljGeneTrain.set_index('Sample')
AlljGeneTrainsmall.index = AlljGeneTrainsmall.index.str.replace("HIP", "")

AllvGeneTestsmall = AllvGeneTest.set_index('Sample')
AllvGeneTestsmall.index = AllvGeneTestsmall.index.str.replace("HIP", "")

AlljGeneTestsmall = AlljGeneTest.set_index('Sample')
AlljGeneTestsmall.index = AlljGeneTestsmall.index.str.replace("HIP", "")



trainSetSamples
HIP08702
trainSetSamples
HIP14015
trainSetSamples
HIP16867
trainSetSamples
HIP01597
trainSetSamples
HIP19089
trainSetSamples
HIP02790
trainSetSamples
HIP05535
trainSetSamples
HIP00602
trainSetSamples
HIP03502
trainSetSamples
HIP02928
trainSetSamples
HIP14048
trainSetSamples
HIP14244
trainSetSamples
HIP13183
trainSetSamples
HIP11553
trainSetSamples
HIP13370
trainSetSamples
HIP03107
trainSetSamples
HIP02805
trainSetSamples
HIP13722
trainSetSamples
HIP01856
trainSetSamples
HIP14037
trainSetSamples
HIP15854
trainSetSamples
HIP09365
trainSetSamples
HIP14028
trainSetSamples
HIP00715
trainSetSamples
HIP14074
trainSetSamples
HIP03592
trainSetSamples
HIP14223


/usr/wisdom/python/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


trainSetSamples
HIP13887
trainSetSamples
HIP17887
trainSetSamples
HIP13966
trainSetSamples
HIP08507
trainSetSamples
HIP09775
trainSetSamples
HIP13661
trainSetSamples
HIP13360
trainSetSamples
HIP05524
trainSetSamples
HIP04532
trainSetSamples
HIP14243
trainSetSamples
HIP05467
trainSetSamples
HIP13988
trainSetSamples
HIP13833
trainSetSamples
HIP09789
trainSetSamples
HIP17837
trainSetSamples
HIP17737
trainSetSamples
HIP04611
trainSetSamples
HIP00773
trainSetSamples
HIP14174
trainSetSamples
HIP13636
trainSetSamples
HIP12143
trainSetSamples
HIP13856
trainSetSamples
HIP14018
trainSetSamples
HIP04545
trainSetSamples
HIP10669
trainSetSamples
HIP01266
trainSetSamples
HIP01765
trainSetSamples
HIP08596
trainSetSamples
HIP13294
trainSetSamples
HIP13809
trainSetSamples
HIP01313
trainSetSamples
HIP05398
trainSetSamples
HIP05540
trainSetSamples
HIP10377
trainSetSamples
HIP08345
trainSetSamples
HIP02103
trainSetSamples
HIP13951


/usr/wisdom/python/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


trainSetSamples
HIP00838
trainSetSamples
HIP13848
trainSetSamples
HIP01055
trainSetSamples
HIP13992
trainSetSamples
HIP02873
trainSetSamples
HIP14059
trainSetSamples
HIP13949
trainSetSamples
HIP00769
trainSetSamples
HIP13515
trainSetSamples
HIP01805
trainSetSamples
HIP14217
trainSetSamples
HIP10846
trainSetSamples
HIP01392


...

TestSetSamples
HIP02997
TestSetSamples
HIP08816
TestSetSamples
HIP13975
TestSetSamples
HIP14022
TestSetSamples
HIP05841
TestSetSamples
HIP03233
TestSetSamples
HIP14209
TestSetSamples
HIP05563
TestSetSamples
HIP13853
TestSetSamples
HIP09253
TestSetSamples
HIP14041
TestSetSamples
HIP01836
TestSetSamples
HIP03197
TestSetSamples
HIP13263
TestSetSamples
HIP12538
TestSetSamples
HIP13847
TestSetSamples
HIP14064
TestSetSamples
HIP14060
TestSetSamples
HIP08076
TestSetSamples
HIP11937
TestSetSamples
HIP15855
TestSetSamples
HIP13869
TestSetSamples
HIP01571
TestSetSamples
HIP14202
TestSetSamples
HIP09001
TestSetSamples
HIP17534
TestSetSamples
HIP02848
TestSetSamples
HIP01181
TestSetSamples
HIP03591
TestSetSamples
HIP13478
TestSetSamples
HIP13710
TestSetSamples
HIP13157
TestSetSamples
HIP00169
TestSetSamples
HIP05533
TestSetSamples
HIP00851
TestSetSamples
HIP05595
TestSetSamples
HIP09284
TestSetSamples
HIP03651
TestSetSamples
HIP01797
TestSetSamples
HIP17454
TestSetSamples
HIP09097
TestSetSamples
H

/usr/wisdom/python/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,9,10,11,18,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


TestSetSamples
HIP02811
TestSetSamples
HIP10602
TestSetSamples
HIP00934
TestSetSamples
HIP14130
TestSetSamples
HIP14227
TestSetSamples
HIP13265
TestSetSamples
HIP13877
TestSetSamples
HIP13355
TestSetSamples
HIP10376
TestSetSamples
HIP10445
TestSetSamples
HIP13015
TestSetSamples
HIP13244
TestSetSamples
HIP10823
TestSetSamples
HIP13794
TestSetSamples
HIP13233
TestSetSamples
HIP02024
TestSetSamples
HIP13947
TestSetSamples
HIP14034
TestSetSamples
HIP13554
TestSetSamples
HIP03814
TestSetSamples
HIP13987
TestSetSamples
HIP01140
TestSetSamples
HIP10746
TestSetSamples
HIP10514
TestSetSamples
HIP17445
TestSetSamples
HIP03228
TestSetSamples
HIP13782
TestSetSamples
HIP01582
TestSetSamples
HIP01384
TestSetSamples
HIP10447
TestSetSamples
HIP03385
TestSetSamples
HIP03677
TestSetSamples
HIP11649
TestSetSamples
HIP13880
TestSetSamples
HIP08439
TestSetSamples
HIP13831
TestSetSamples
HIP02663
TestSetSamples
HIP10358
TestSetSamples
HIP14176
TestSetSamples
HIP13796
TestSetSamples
HIP09159
TestSetSamples
H

/usr/wisdom/python/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,10,11,13,18,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


TestSetSamples
HIP01795
TestSetSamples
HIP14095
TestSetSamples
HIP09026
TestSetSamples
HIP17657
TestSetSamples
HIP13627
TestSetSamples
HIP14036
TestSetSamples
HIP14121
TestSetSamples
HIP13956
TestSetSamples
HIP19717
TestSetSamples
HIP13709
TestSetSamples
HIP17585
TestSetSamples
HIP13945
TestSetSamples
HIP12703


In [13]:
filename1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/AlljGeneTestsmall_293'
AlljGeneTestsmall.to_pickle(filename1)

filename2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/AllvGeneTestsmall_293'
AllvGeneTestsmall.to_pickle(filename2)

filename3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/AlljGeneTrainsmall_294'
AlljGeneTrainsmall.to_pickle(filename3)

filename4='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/AllvGeneTrainsmall_294'
AllvGeneTrainsmall.to_pickle(filename4)

In [ ]:
# calculate probs for each v and j gene in the train and test sets (note - 'unresolved' sequences are included!!!)

In [14]:
jGeneTestProbs=AlljGeneTestsmall['jGeneName'].value_counts(normalize=True)

In [15]:
jGeneTrainProbs=AlljGeneTrainsmall['jGeneName'].value_counts(normalize=True)

In [16]:
vGeneTestProbs=AllvGeneTestsmall['vGeneName'].value_counts(normalize=True)

In [17]:
vGeneTrainProbs=AllvGeneTrainsmall['vGeneName'].value_counts(normalize=True)

In [18]:
filename1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/jGeneTestProbs_293'
jGeneTestProbs.to_pickle(filename1)

filename2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/vGeneTestProbs_293'
vGeneTestProbs.to_pickle(filename2)

filename3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/jGeneTrainProbs_294'
jGeneTrainProbs.to_pickle(filename3)

filename4='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/vGeneTrainProbs_294'
vGeneTrainProbs.to_pickle(filename4)

In [21]:
# generate ins1 and ins2 sequences list for the train and test sets:

def genInsSeqDF_definedSamples(sampleList, toSave,insType):
    
    
    insSeq_sample_df_list = []
    count = 1
    for sample_name in sampleList:
        print count
        if insType==1:
            insSeq_sample_df = genIns1SeqDF(sample_name)
        if insType==2:
            insSeq_sample_df = genIns2SeqDF(sample_name)
        insSeq_sample_df['Sample'] = sample_name
        insSeq_sample_df_list.append(insSeq_sample_df)
        # print len(insSeq_sample_df)
        count += 1
    
    AllinsSeqsDF = pd.concat(insSeq_sample_df_list)
    print len(AllinsSeqsDF)
    AllinsSeqsDFsmall = AllinsSeqsDF.set_index('Sample')
    AllinsSeqsDFsmall.index = AllinsSeqsDFsmall.index.str.replace("HIP", "")
    
    
    if toSave:
        with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/AllinsSeqsDFsmall , "wb") as f:
            pickle.dump(AllinsSeqsDFsmall, f)
        f.close()
        writer = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/AllinsSeqsDFsmall.xlsx' 
        AllinsSeqsDFsmall.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)  # # saving the correct Reg table to pickles and excel:
    
    return AllinsSeqsDFsmall






In [22]:
## change to 'TRUE' before running again!!!

AllinsSeqsDFsmall_ins1_train=genInsSeqDF_definedSamples(trainSetSamples, False,1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


/usr/wisdom/python/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


/usr/wisdom/python/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


65
66
67
68
69
70
71
72
73
74
75
76
77


/usr/wisdom/python/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9,24) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
12391729


In [60]:


AllinsSeqsDFsmall_ins1_train.tail()

,length,sequence
Sample,,
14129,3,CGG
14129,4,TCCC
14129,24,ACGTGGGTTTATCCTGGCGTAGGA
14129,14,CCGCGCGGCCCTTG
14129,1,T


In [23]:
## change to 'TRUE' before running again!!!

AllinsSeqsDFsmall_ins2_train=genInsSeqDF_definedSamples(trainSetSamples, False,2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [24]:
## change to 'TRUE' before running again!!!

AllinsSeqsDFsmall_ins1_test=genInsSeqDF_definedSamples(TestSetSamples, False,1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164


/usr/wisdom/python/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,9,10,11,18,24) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280


/usr/wisdom/python/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9,10,11,13,18,24) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


281
282
283
284
285
286
287
288
289
290
291
292
293
12105091


In [25]:
## change to 'TRUE' before running again!!!

AllinsSeqsDFsmall_ins2_test=genInsSeqDF_definedSamples(TestSetSamples, False,2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [26]:
threshLength=20

In [27]:
dinucNormDFDict_train_ins1,nt1FreqsDictDict_train_ins1,lengthCount_train_ins1=genSeqProbsForLengths(AllinsSeqsDFsmall_ins1_train, threshLength)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [33]:
dinucNormDFDict_train_ins2

NameError: name 'dinucNormDFDict_train_ins2' is not defined

In [34]:
dinucNormDFDict_train_ins2,nt1FreqsDictDict_train_ins2,lengthCount_train_ins2=genSeqProbsForLengths(AllinsSeqsDFsmall_ins2_train, threshLength)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [ ]:
dinucNormDFDict_test_ins1,nt1FreqsDictDict_test_ins1,lengthCount_test_ins1=genSeqProbsForLengths(AllinsSeqsDFsmall_ins1_test, threshLength)

In [ ]:
dinucNormDFDict_test_ins2,nt1FreqsDictDict_test_ins2,lengthCount_test_ins2=genSeqProbsForLengths(AllinsSeqsDFsmall_ins2_test, threshLength)

In [ ]:
#save all to pickles:

In [32]:
#ins1_train:

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/dinucNormDFDict_train_ins1', 'wb') as file1:
    pickle.dump(dinucNormDFDict_train_ins1, file1)
file1.close()

with open ('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/nt1FreqsDictDict_train_ins1', 'wb') as file2:
    pickle.dump(nt1FreqsDictDict_train_ins1, file2)
file2.close()

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/lengthCount_train_ins1' , 'wb') as file3:
     pickle.dump(lengthCount_train_ins1, file3)
file3.close()
    

In [35]:
#ins2_train:

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/dinucNormDFDict_train_ins2', 'wb') as file1:
    pickle.dump(dinucNormDFDict_train_ins2, file1)
file1.close()

with open ('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/nt1FreqsDictDict_train_ins2' , 'wb') as file2:
    pickle.dump(nt1FreqsDictDict_train_ins2, file2)
file2.close()

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/lengthCount_train_ins2', 'wb') as file3:
     pickle.dump(lengthCount_train_ins2, file3)
file3.close()
    

In [ ]:
#ins1_test:

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/dinucNormDFDict_test_ins1', 'wb') as file1:
    pickle.dump(dinucNormDFDict_test_ins1, file1)
file1.close()

with open ('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/nt1FreqsDictDict_test_ins1' , 'wb') as file2:
    pickle.dump(nt1FreqsDictDict_test_ins1, file2)
file2.close()

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/lengthCount_test_ins1' , 'wb') as file3:
     pickle.dump(lengthCount_test_ins1, file3)
file3.close()
    

In [ ]:
#ins2_test:

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/dinucNormDFDict_test_ins2', 'wb') as file1:
    pickle.dump(dinucNormDFDict_test_ins2, file1)
file1.close()

with open ('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/nt1FreqsDictDict_test_ins2' , 'wb') as file2:
    pickle.dump(nt1FreqsDictDict_test_ins2, file2)
file2.close()

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/lengthCount_test_ins2' , 'wb') as file3:
     pickle.dump(lengthCount_test_ins2, file3)
file3.close()
    

In [ ]:
#continue with dependent probabilities  

In [78]:
#generate combined list of D and J for the train and test sets:

DJ_GeneTrainSetDF_list = []
DJ_GeneTestSetDF_list = []

count_train=1    
for sample_name in trainSetSamples:
    
    print 'trainSetSamples...'
    print count_train, sample_name
    
    sample_df = pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/%s.tsv" % sample_name) 
    sample_df_non_prod = sample_df[sample_df['sequenceStatus'] != 'In']
    
        
    DJ_GeneTrainSetDF = pd.DataFrame(sample_df_non_prod[['jGeneName','dFamilyName']])
    DJ_GeneTrainSetDF['Sample'] = sample_name
    DJ_GeneTrainSetDF_list.append(DJ_GeneTrainSetDF)
    
    count_train+=1
      
count_test=1    
for sample_name in TestSetSamples:
    
    print 'TestSetSamples...'
    print count_test, sample_name    
    
    sample_df = pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/%s.tsv" % sample_name) 
    sample_df_non_prod = sample_df[sample_df['sequenceStatus'] != 'In']

    DJ_GeneTestSetDF = pd.DataFrame(sample_df_non_prod[['jGeneName','dFamilyName']])
    DJ_GeneTestSetDF['Sample'] = sample_name
    DJ_GeneTestSetDF_list.append(DJ_GeneTestSetDF)
    
    count_test+=1
    

All_DJ_GeneTrain = pd.concat(DJ_GeneTrainSetDF_list)
All_DJ_GeneTest = pd.concat(DJ_GeneTestSetDF_list)


#print len(AllvGeneDFs)

All_DJ_GeneTrain = All_DJ_GeneTrain.set_index('Sample')
All_DJ_GeneTrain.index = All_DJ_GeneTrain.index.str.replace("HIP", "")

All_DJ_GeneTest = All_DJ_GeneTest.set_index('Sample')
All_DJ_GeneTest.index = All_DJ_GeneTest.index.str.replace("HIP", "")



trainSetSamples...
HIP13220
trainSetSamples...
HIP03812
trainSetSamples...
HIP02024
trainSetSamples...
HIP13352
trainSetSamples...
HIP04480
trainSetSamples...
HIP12538
trainSetSamples...
HIP03597
trainSetSamples...
HIP11058
trainSetSamples...
HIP13956
trainSetSamples...
HIP03484
trainSetSamples...
HIP09118
trainSetSamples...
HIP10846
trainSetSamples...
HIP03720
trainSetSamples...
HIP00775
trainSetSamples...
HIP01206
trainSetSamples...
HIP14020
trainSetSamples...
HIP13505
trainSetSamples...
HIP09235
trainSetSamples...
HIP13774
trainSetSamples...
HIP13903
trainSetSamples...
HIP10358
trainSetSamples...
HIP01765
trainSetSamples...
HIP14004
trainSetSamples...
HIP00602
trainSetSamples...
HIP10376
trainSetSamples...
HIP13252
trainSetSamples...
HIP10568
trainSetSamples...
HIP12097
trainSetSamples...
HIP08989
trainSetSamples...
HIP05331
trainSetSamples...
HIP13228
trainSetSamples...
HIP03592
trainSetSamples...
HIP05960
trainSetSamples...
HIP14060
trainSetSamples...
HIP03618
trainSetSamples...
H

trainSetSamples...
HIP14129
TestSetSamples...
HIP03385
TestSetSamples...
HIP01266
TestSetSamples...
HIP13703
TestSetSamples...
HIP01162
TestSetSamples...
HIP13996
TestSetSamples...
HIP01180
TestSetSamples...
HIP08596
TestSetSamples...
HIP13209
TestSetSamples...
HIP13777
TestSetSamples...
HIP08337
TestSetSamples...
HIP05377
TestSetSamples...
HIP01313
TestSetSamples...
HIP13265
TestSetSamples...
HIP14214
TestSetSamples...
HIP13214
TestSetSamples...
HIP14234
TestSetSamples...
HIP13992
TestSetSamples...
HIP13887
TestSetSamples...
HIP04555
TestSetSamples...
HIP14160
TestSetSamples...
HIP13360
TestSetSamples...
HIP12034
TestSetSamples...
HIP04532
TestSetSamples...
HIP13361
TestSetSamples...
HIP02928
TestSetSamples...
HIP02737
TestSetSamples...
HIP14041
TestSetSamples...
HIP02271
TestSetSamples...
HIP13859
TestSetSamples...
HIP01004
TestSetSamples...
HIP05409
TestSetSamples...
HIP14134
TestSetSamples...
HIP09020
TestSetSamples...
HIP15855
TestSetSamples...
HIP12165
TestSetSamples...
HIP01391


In [79]:
# save DJ dfs to pickles:

filename1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/All_DJ_GeneTrainsmall_293'
All_DJ_GeneTrain.to_pickle(filename1)

filename3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/All_DJ_GeneTest_294'
All_DJ_GeneTest.to_pickle(filename3)



In [82]:
# calculate conditional probs for each d and j pair in the train and test sets (note - 'unresolved' sequences are included!!!)
indPar='jGeneName'
depPar='dFamilyName'
df=All_DJ_GeneTrain

p_DgivenJ_Train=pd.crosstab(index=df[indPar], columns=df[depPar], normalize='index')

indPar='jGeneName'
depPar='dFamilyName'
df=All_DJ_GeneTest

p_DgivenJ_Test=pd.crosstab(index=df[indPar], columns=df[depPar], normalize='index')

In [83]:
p_DgivenJ_Test

dFamilyName,TCRBD01,TCRBD02
jGeneName,,
TCRBJ01-01,0.650991,0.349009
TCRBJ01-02,0.681985,0.318015
TCRBJ01-03,0.654216,0.345784
TCRBJ01-04,0.663660,0.336340
TCRBJ01-05,0.677254,0.322746
TCRBJ01-06,0.673408,0.326592
TCRBJ02-01,0.242430,0.757570
TCRBJ02-02,0.377026,0.622974
TCRBJ02-03,0.257722,0.742278


In [84]:
p_DgivenJ_Train

dFamilyName,TCRBD01,TCRBD02
jGeneName,,
TCRBJ01-01,0.649889,0.350111
TCRBJ01-02,0.682267,0.317733
TCRBJ01-03,0.654722,0.345278
TCRBJ01-04,0.662055,0.337945
TCRBJ01-05,0.676670,0.323330
TCRBJ01-06,0.672566,0.327434
TCRBJ02-01,0.241629,0.758371
TCRBJ02-02,0.375777,0.624223
TCRBJ02-03,0.256299,0.743701


In [85]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/p_DgivenJ_Train'
p_DgivenJ_Train.to_pickle(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/p_DgivenJ_Test'
p_DgivenJ_Test.to_pickle(file2)

## define a function to calculate and save all conditional probs

In [19]:
def calc_conditional_probs(indPar,depPar):
    
    print 'generating parameters lists for train and test sets:'
    condProb_TrainSetDF_list = []
    condProb_TestSetDF_list = []

    count_train=1    
    for sample_name in trainSetSamples:

        print '%s %s %s train...' %(indPar, depPar,  count_train)
        
        sample_df = pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/%s.tsv" % sample_name) 
        sample_df_non_prod = sample_df[sample_df['sequenceStatus'] != 'In']


        condProb_TrainSetDF = pd.DataFrame(sample_df_non_prod[[indPar,depPar]])
        condProb_TrainSetDF['Sample'] = sample_name
        condProb_TrainSetDF_list.append(condProb_TrainSetDF)

        count_train+=1

    count_test=1    
    for sample_name in TestSetSamples:

        print '%s %s %s test...' %(indPar, depPar,  count_test)

        sample_df = pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/%s.tsv" % sample_name) 
        sample_df_non_prod = sample_df[sample_df['sequenceStatus'] != 'In']

        condProb_TestSetDF = pd.DataFrame(sample_df_non_prod[[indPar,depPar]])
        condProb_TestSetDF['Sample'] = sample_name
        condProb_TestSetDF_list.append(condProb_TestSetDF)

        count_test+=1


    All_condProb_Train = pd.concat(condProb_TrainSetDF_list)
    All_condProb_Test = pd.concat(condProb_TestSetDF_list)
    
    
  


    #print len(AllvGeneDFs)

    All_condProb_Train = All_condProb_Train.set_index('Sample')
    All_condProb_Train.index = All_condProb_Train.index.str.replace("HIP", "")

    All_condProb_Test = All_condProb_Test.set_index('Sample')
    All_condProb_Test.index = All_condProb_Test.index.str.replace("HIP", "")
    
    
    print 'saving parameter lists for train and test sets... '
    
    filename1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/parms_values_%s_%s_Train_293' %(indPar, depPar)
    All_condProb_Train.to_pickle(filename1)

    filename2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/parms_values_%s_%s_Test_294' %(indPar, depPar)
    All_condProb_Test.to_pickle(filename2)
    
    # calculate conditional probs for each pair in the train and test sets (note - 'unresolved' sequences are included!!!)
    
    print 'calculating conditional probs for %s and %s' %(indPar, depPar)
    Train_df=All_condProb_Train
    Test_df=All_condProb_Test

    condProbs_Train=pd.crosstab(index=Train_df[indPar], columns=Train_df[depPar], normalize='index')
    condProbs_Test=pd.crosstab(index=Test_df[indPar], columns=Test_df[depPar], normalize='index')
    
    file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_%s_%s_Train_293' %(indPar, depPar)
    condProbs_Train.to_pickle(file3)

    file4='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/SuffStat/WholeCDR3SeqPred/condProb_%s_%s_Test_294' %(indPar, depPar)
    condProbs_Test.to_pickle(file4)

    return condProbs_Train,condProbs_Test


In [20]:
indPar_list=['jGeneName','vGeneName','dFamilyName','dFamilyName','jGeneName']
depPar_list=['dFamilyName','vDeletion','d5Deletion','d3Deletion','jDeletion']

for n in range(5):
    indPar=indPar_list[n]
    depPar=depPar_list[n]
    print indPar,depPar
    condProbs_Train,condProbs_Test=calc_conditional_probs(indPar,depPar)
    

jGeneName dFamilyName
generating parameters lists for train and test sets:
jGeneName dFamilyName 1 train...
jGeneName dFamilyName 2 train...
jGeneName dFamilyName 3 train...
jGeneName dFamilyName 4 train...
jGeneName dFamilyName 5 train...
jGeneName dFamilyName 6 train...
jGeneName dFamilyName 7 train...
jGeneName dFamilyName 8 train...
jGeneName dFamilyName 9 train...
jGeneName dFamilyName 10 train...
jGeneName dFamilyName 11 train...
jGeneName dFamilyName 12 train...
jGeneName dFamilyName 13 train...
jGeneName dFamilyName 14 train...
jGeneName dFamilyName 15 train...
jGeneName dFamilyName 16 train...
jGeneName dFamilyName 17 train...
jGeneName dFamilyName 18 train...
jGeneName dFamilyName 19 train...
jGeneName dFamilyName 20 train...
jGeneName dFamilyName 21 train...
jGeneName dFamilyName 22 train...
jGeneName dFamilyName 23 train...
jGeneName dFamilyName 24 train...
jGeneName dFamilyName 25 train...
jGeneName dFamilyName 26 train...
jGeneName dFamilyName 27 train...


/usr/wisdom/python/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


jGeneName dFamilyName 28 train...
jGeneName dFamilyName 29 train...
jGeneName dFamilyName 30 train...
jGeneName dFamilyName 31 train...
jGeneName dFamilyName 32 train...
jGeneName dFamilyName 33 train...
jGeneName dFamilyName 34 train...
jGeneName dFamilyName 35 train...
jGeneName dFamilyName 36 train...
jGeneName dFamilyName 37 train...
jGeneName dFamilyName 38 train...
jGeneName dFamilyName 39 train...
jGeneName dFamilyName 40 train...
jGeneName dFamilyName 41 train...
jGeneName dFamilyName 42 train...
jGeneName dFamilyName 43 train...
jGeneName dFamilyName 44 train...
jGeneName dFamilyName 45 train...
jGeneName dFamilyName 46 train...
jGeneName dFamilyName 47 train...
jGeneName dFamilyName 48 train...
jGeneName dFamilyName 49 train...
jGeneName dFamilyName 50 train...
jGeneName dFamilyName 51 train...
jGeneName dFamilyName 52 train...
jGeneName dFamilyName 53 train...
jGeneName dFamilyName 54 train...
jGeneName dFamilyName 55 train...
jGeneName dFamilyName 56 train...
jGeneName dFam

/usr/wisdom/python/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


jGeneName dFamilyName 65 train...
jGeneName dFamilyName 66 train...
jGeneName dFamilyName 67 train...
jGeneName dFamilyName 68 train...
jGeneName dFamilyName 69 train...
jGeneName dFamilyName 70 train...
jGeneName dFamilyName 71 train...
jGeneName dFamilyName 72 train...
jGeneName dFamilyName 73 train...
jGeneName dFamilyName 74 train...
jGeneName dFamilyName 75 train...
jGeneName dFamilyName 76 train...
jGeneName dFamilyName 77 train...


...

dFamilyName d3Deletion 18 train...
dFamilyName d3Deletion 19 train...
dFamilyName d3Deletion 20 train...
dFamilyName d3Deletion 21 train...
dFamilyName d3Deletion 22 train...
dFamilyName d3Deletion 23 train...
dFamilyName d3Deletion 24 train...
dFamilyName d3Deletion 25 train...
dFamilyName d3Deletion 26 train...
dFamilyName d3Deletion 27 train...
dFamilyName d3Deletion 28 train...
dFamilyName d3Deletion 29 train...
dFamilyName d3Deletion 30 train...
dFamilyName d3Deletion 31 train...
dFamilyName d3Deletion 32 train...
dFamilyName d3Deletion 33 train...
dFamilyName d3Deletion 34 train...
dFamilyName d3Deletion 35 train...
dFamilyName d3Deletion 36 train...
dFamilyName d3Deletion 37 train...
dFamilyName d3Deletion 38 train...
dFamilyName d3Deletion 39 train...
dFamilyName d3Deletion 40 train...
dFamilyName d3Deletion 41 train...
dFamilyName d3Deletion 42 train...
dFamilyName d3Deletion 43 train...
dFamilyName d3Deletion 44 train...
dFamilyName d3Deletion 45 train...
dFamilyName d3Deleti

dFamilyName d3Deletion 248 train...
dFamilyName d3Deletion 249 train...
dFamilyName d3Deletion 250 train...
dFamilyName d3Deletion 251 train...
dFamilyName d3Deletion 252 train...
dFamilyName d3Deletion 253 train...
dFamilyName d3Deletion 254 train...
dFamilyName d3Deletion 255 train...
dFamilyName d3Deletion 256 train...
dFamilyName d3Deletion 257 train...
dFamilyName d3Deletion 258 train...
dFamilyName d3Deletion 259 train...
dFamilyName d3Deletion 260 train...
dFamilyName d3Deletion 261 train...
dFamilyName d3Deletion 262 train...
dFamilyName d3Deletion 263 train...
dFamilyName d3Deletion 264 train...
dFamilyName d3Deletion 265 train...
dFamilyName d3Deletion 266 train...
dFamilyName d3Deletion 267 train...
dFamilyName d3Deletion 268 train...
dFamilyName d3Deletion 269 train...
dFamilyName d3Deletion 270 train...
dFamilyName d3Deletion 271 train...
dFamilyName d3Deletion 272 train...
dFamilyName d3Deletion 273 train...
dFamilyName d3Deletion 274 train...
dFamilyName d3Deletion 275 t

dFamilyName d3Deletion 190 test...
dFamilyName d3Deletion 191 test...
dFamilyName d3Deletion 192 test...
dFamilyName d3Deletion 193 test...
dFamilyName d3Deletion 194 test...
dFamilyName d3Deletion 195 test...
dFamilyName d3Deletion 196 test...
dFamilyName d3Deletion 197 test...
dFamilyName d3Deletion 198 test...
dFamilyName d3Deletion 199 test...
dFamilyName d3Deletion 200 test...
dFamilyName d3Deletion 201 test...
dFamilyName d3Deletion 202 test...
dFamilyName d3Deletion 203 test...
dFamilyName d3Deletion 204 test...
dFamilyName d3Deletion 205 test...
dFamilyName d3Deletion 206 test...
dFamilyName d3Deletion 207 test...
dFamilyName d3Deletion 208 test...
dFamilyName d3Deletion 209 test...
dFamilyName d3Deletion 210 test...
dFamilyName d3Deletion 211 test...
dFamilyName d3Deletion 212 test...
dFamilyName d3Deletion 213 test...
dFamilyName d3Deletion 214 test...
dFamilyName d3Deletion 215 test...
dFamilyName d3Deletion 216 test...
dFamilyName d3Deletion 217 test...
dFamilyName d3Deleti

jGeneName jDeletion 137 train...
jGeneName jDeletion 138 train...
jGeneName jDeletion 139 train...
jGeneName jDeletion 140 train...
jGeneName jDeletion 141 train...
jGeneName jDeletion 142 train...
jGeneName jDeletion 143 train...
jGeneName jDeletion 144 train...
jGeneName jDeletion 145 train...
jGeneName jDeletion 146 train...
jGeneName jDeletion 147 train...
jGeneName jDeletion 148 train...
jGeneName jDeletion 149 train...
jGeneName jDeletion 150 train...
jGeneName jDeletion 151 train...
jGeneName jDeletion 152 train...
jGeneName jDeletion 153 train...
jGeneName jDeletion 154 train...
jGeneName jDeletion 155 train...
jGeneName jDeletion 156 train...
jGeneName jDeletion 157 train...
jGeneName jDeletion 158 train...
jGeneName jDeletion 159 train...
jGeneName jDeletion 160 train...
jGeneName jDeletion 161 train...
jGeneName jDeletion 162 train...
jGeneName jDeletion 163 train...
jGeneName jDeletion 164 train...
jGeneName jDeletion 165 train...
jGeneName jDeletion 166 train...
jGeneName 

jGeneName jDeletion 98 test...
jGeneName jDeletion 99 test...
jGeneName jDeletion 100 test...
jGeneName jDeletion 101 test...
jGeneName jDeletion 102 test...
jGeneName jDeletion 103 test...
jGeneName jDeletion 104 test...
jGeneName jDeletion 105 test...
jGeneName jDeletion 106 test...
jGeneName jDeletion 107 test...
jGeneName jDeletion 108 test...
jGeneName jDeletion 109 test...
jGeneName jDeletion 110 test...
jGeneName jDeletion 111 test...
jGeneName jDeletion 112 test...
jGeneName jDeletion 113 test...
jGeneName jDeletion 114 test...
jGeneName jDeletion 115 test...
jGeneName jDeletion 116 test...
jGeneName jDeletion 117 test...
jGeneName jDeletion 118 test...
jGeneName jDeletion 119 test...
jGeneName jDeletion 120 test...
jGeneName jDeletion 121 test...
jGeneName jDeletion 122 test...
jGeneName jDeletion 123 test...
jGeneName jDeletion 124 test...
jGeneName jDeletion 125 test...
jGeneName jDeletion 126 test...
jGeneName jDeletion 127 test...
jGeneName jDeletion 128 test...
jGeneName 

p(CDR3)=logP(V)+logP(D|J)+logP(J)+logP(delv|v)+logP(delD5|D)+logP(delD3|D)+logP(delJ|J)+logP(ins1)+logP(ins2)

for each sequence:
(1) get identities of each recombination event
(2) calculate the probs for each event, and sum their logs
(3)calculate in how many samples we will expect this sequence to appear 